In [1]:
########## auto login ############
import login
from login import *
import pandas as pd
from loguru import logger
import pytz
from time import sleep
import datetime
import json
import pandas as pd
logger.add("./logs/MNC_logs_{time}.log")

access token : Jn5qqO9z8NzEn6OOSHNs5d5DIuCUZzHT
request token : QWj7Hif46w9Gn9XoOEuTm2sdZq1OCMDX


1

In [2]:
############# Import Old qty_df File ####################
import os.path
from os import path
is_qty_df_available = path.exists('qty_df.p')
old_qty_df = pd.DataFrame()
if(is_qty_df_available):
    old_qty_df = joblib.load('./qty_df.p')
# old_qty_df

In [3]:
############## Parameters #######################
my_margin = 400
order_type = 'MIS'
exchange_type = kite.EXCHANGE_NSE
offset_quantity = 0
away_from_circuit = 0.50
########### Constants ##########
n50_token = 256265
n50_symbol = 'NIFTY 50'
days_ago=1

In [6]:
###### Get Stocks Margin Map ############
import requests
url = 'https://api.kite.trade/margins/equity'
margin_data = requests.get(url, timeout=2.50)
margin_map = json.loads(margin_data.content.decode('utf-8'))
margin_map_df = pd.DataFrame(margin_map)

############ Get N/50 Stocks List ###########
n50df = pd.read_csv('n50.csv')
n50df = n50df.loc[n50df['tradable'] == 'y']

############ Merge Both Data ################
merged_df = pd.merge(n50df, margin_map_df, on="tradingsymbol")
merged_df

,Sr.,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,...,segment,exchange,tradable,weight,margin,co_lower,mis_multiplier,co_upper,nrml_margin,mis_margin
0,1.0,3861249.0,15083.0,ADANIPORTS,ADANI PORT & SEZ,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.52,0,12.00,5.0,8.0,0,20.00
1,2.0,60417.0,236.0,ASIANPAINT,ASIAN PAINTS,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,1.99,0,4.99,12.0,20.0,0,8.31
2,3.0,1510401.0,5900.0,AXISBANK,AXIS BANK,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,2.43,0,12.00,5.0,8.0,0,20.00
3,4.0,4267265.0,16669.0,BAJAJ-AUTO,BAJAJ AUTO,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.75,0,5.21,12.0,19.0,0,8.68
4,5.0,4268801.0,16675.0,BAJAJFINSV,BAJAJ FINSERV.,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.67,0,12.00,5.0,8.0,0,20.00
5,6.0,81153.0,317.0,BAJFINANCE,BAJAJ FINANCE,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,1.75,0,12.00,5.0,8.0,0,20.00
6,7.0,2714625.0,10604.0,BHARTIARTL,BHARTI AIRTEL,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,2.08,0,12.00,5.0,8.0,0,20.00
7,8.0,134657.0,526.0,BPCL,BHARAT PETROLEUM CORP LT,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.57,0,13.21,5.0,8.0,0,22.01
8,9.0,140033.0,547.0,BRITANNIA,BRITANNIA INDUSTRIES,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.82,0,5.50,11.0,18.0,0,9.17
9,10.0,177665.0,694.0,CIPLA,CIPLA,0.0,NaN,0.0,0.05,1.0,...,NSE,NSE,y,0.76,0,5.50,11.0,18.0,0,9.17


In [5]:
############## Stock Margin Calucator for Bracket Order #####################
def calc_margin(co_upper,co_lower,price,stoploss=100, transaction_type = 'sell',quantity=1):
    '''calc_margin(co_upper,co_lower,stk_data,price,stoploss=10, transaction_type = 'sell',quantity=1)'''
    co_lower = co_lower/100
    co_upper = co_upper/100
    trigger = price - (co_upper * price)
    
    if stoploss < trigger:
        stoploss = trigger
    else:
        trigger = stoploss
    x = 0
    if transaction_type == 'buy':
        x = (price - trigger) * quantity
    else:
        x = (trigger - price) * quantity
        
    y = co_lower * price * quantity
    margin  = x if( x > y) else y
    margin = margin + (margin * 0.2)
    return margin

############# Get 9:15 Data ##############################################
def get_mornign_data( inst_token, interval='5minute'):
    '''get_mornign_data( inst_token, interval='5minute')'''
    interval=interval
    inst_token = inst_token
    todaydt=datetime.date.today()
    hud_ago=todaydt-datetime.timedelta(days=1)
    to_date=datetime.date.isoformat(todaydt)

    from_date=datetime.date.isoformat(hud_ago)
    data = kite.historical_data(inst_token, from_date, to_date, interval, continuous=False)

    mydata = pd.DataFrame(data)
    today = datetime.date.today()

#     print('Today Date : '+str(today))
    yest_date =  mydata[-days_ago:][['date']].to_numpy()[0][0]
    last_close =  mydata[-days_ago:][['close']].to_numpy()[0][0]
    last_high =  mydata[-days_ago:][['high']].to_numpy()[0][0]
    last_low =  mydata[-days_ago:][['low']].to_numpy()[0][0]
#     print('yest_date : '+str(yest_date))
#     print('last close : '+ str(last_close))
#     print('last high : '+ str(last_high))
#     print('last low : '+ str(last_low))

    import pytz
    tz = pytz.timezone("Asia/Calcutta")
    b =tz.localize( datetime.datetime(today.year, today.month, 20, 9, 15,0,0))
    # today.day
    timestamp = pd.Timestamp(b)

    mycandle = mydata.loc[mydata['date'] == timestamp]
    return {'open':mycandle['open'].to_numpy()[0],
            'high':mycandle['high'].to_numpy()[0],
            'low':mycandle['low'].to_numpy()[0],
            'close':mycandle['close'].to_numpy()[0],
           }

########### Decide Buy/Sell ###############
def decide_buy_sell(morning_candle):
    direction = ''
    if(morning_candle['open']>morning_candle['close']):
        direction = 'sell'
    if(morning_candle['open']<morning_candle['close']):
        direction = 'buy'
    return direction

In [ ]:
# ################ Wait Till 9:20 #####################
# def act(x):
#     return x+10

# def wait_start(runTime, action):
#     startTime = datetime.time(*(map(int, runTime.split(':'))))
#     while startTime > datetime.datetime.today().time(): 
#         sleep(1)
#     return action

# wait_start('9:20', lambda: act(100))

In [ ]:
################# Add Buy/Sell Direction to merged_df ############
direction_list = []
for i in merged_df.instrument_token.to_list():
    this_map = get_mornign_data(i)
    this_dir = decide_buy_sell(this_map)
    direction_list.append({'instrument_token':i,'direction':this_dir,
                           'morning_open':this_map['open'],
                           'morning_high':this_map['high'],
                           'morning_low':this_map['low'],
                           'morning_close':this_map['close']})
    print(i)

direction_list_df = pd.DataFrame(direction_list)
merged_df2 = pd.merge(merged_df, direction_list_df, on="instrument_token")
# merged_df2

In [ ]:
################ Get Considered DF Comparing to N50 Direction ########## 
nifty50_morning_candle = get_mornign_data(n50_token)
nifty_direction = decide_buy_sell(nifty50_morning_candle)
considered_df = merged_df2.loc[merged_df2['direction'] == nifty_direction]
# considered_df

In [ ]:
################### Decide Qty for Every Symbol #####################
total_margin_parts = len(considered_df.instrument_token)
per_symbol_margin = my_margin/total_margin_parts

qty_df_list = []

for inst in considered_df.instrument_token:
    this_co_upper = considered_df.loc[considered_df['instrument_token'] == inst]['co_upper'].to_numpy()[0]
    this_co_lower = considered_df.loc[considered_df['instrument_token'] == inst]['co_lower'].to_numpy()[0]
    this_price = 1.0
    this_sl = 1.0
    if(nifty_direction=='buy'):
        this_price = considered_df.loc[considered_df['instrument_token'] == inst]['morning_high'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] == inst]['morning_low'].to_numpy()[0]
    else:
        this_price = considered_df.loc[considered_df['instrument_token'] == inst]['morning_low'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] == inst]['morning_high'].to_numpy()[0]
    this_marg = calc_margin(this_co_upper,this_co_lower,this_price,stoploss=this_sl,
                            transaction_type = nifty_direction) 
    this_qty = int(per_symbol_margin/this_marg)
    qty_df_list.append({'instrument_token':inst,'co_bo_margin':this_marg,
                        'trade_qty':this_qty,'my_qty':0,'ltp':0.0, 'stop_trading':False,
                        'last_updated_on':str(datetime.date.today())})

qty_df = pd.DataFrame(qty_df_list)
print('Per Symbol Margin : '+str(per_symbol_margin))

qty_df = pd.merge(considered_df,qty_df, on="instrument_token")

if(!old_qty_df.empty):
    qty_df['stop_trading'] = old_qty_df['stop_trading']
    qty_df['last_updated_on'] = old_qty_df['last_updated_on']

# qty_df

In [ ]:
###################### Clear Yesterday True/False ##################################
for my_inst in qty_df.instrument_token:
    this_last_up = str(qty_df.loc[qty_df['instrument_token'] ==
                                                         my_inst]['last_updated_on'].to_numpy()[0]).split('-')[2]
    this_last_up = int(this_last_up)
    this_day = datetime.date.today().day
    if(this_day!=this_last_up):
        qty_df.at[qty_df.index[qty_df['instrument_token'] == my_inst].tolist()[0],'stop_trading']= False
        qty_df.at[qty_df.index[qty_df['instrument_token'] == my_inst].tolist()[0],'last_updated_on']= str(datetime.date.today())
        print('Stop Trading Changed for  : '+str(my_inst))

joblib.dump(qty_df,'./qty_df.p')
qty_df

In [ ]:
# from pivottablejs import pivot_ui
# from IPython.display import HTML
# pvi = pivot_ui(qty_df,outfile_path='./pivottablejs.html')
# pvi

In [ ]:
############# Get My Qty ########################################
myquantity = 0
def getquant():
    global myquantity
    global order_type
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        for j in qty_df.instrument_token.to_list():
            if(allpos[i]['instrument_token']==j and  allpos[i]['product']== 'CO'):
                myquantity = allpos[i]['quantity']
                qty_df.at[qty_df.index[qty_df['instrument_token'] == j].tolist()[0],'my_qty']= myquantity
                this_smb = qty_df.loc[qty_df['instrument_token'] == j]['tradingsymbol'].to_numpy()[0]
                print('My Quantity : '+ this_smb + ' : ' + str(allpos[i]['quantity']))
    joblib.dump(qty_df,'./qty_df.p')
getquant()
################# Place Order #####################################
stopbuy = False
stopsell = False
placing_order = False
def placeneworder(inst_token,quantdiff,trigger_price):
    '''placeneworder(inst_token,quantdiff,trigger_price)'''
    global stopbuy,stopsell,placing_order
    my_symbol_ip = qty_df.loc[qty_df['instrument_token'] == inst_token]['tradingsymbol'].to_numpy()[0]
        
    if(quantdiff>0 and stopbuy == False):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                        exchange=kite.EXCHANGE_NSE, 
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        trigger_price = trigger_price,
                        quantity=abs(quantdiff),
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_CO,
                        product=kite.PRODUCT_CO,)
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'stop_trading']= True
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'last_updated_on']= datetime.date.today()
            getquant()
            joblib.dump(qty_df,'./qty_df.p')
            stopsell = False
        except Exception as e:
            stopbuy = False
            print(e)
            getquant()
        placing_order = False
    if(quantdiff<0 and stopsell == False):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                    exchange=kite.EXCHANGE_NSE, 
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    trigger_price = trigger_price,
                    quantity=abs(quantdiff),
                    order_type=kite.ORDER_TYPE_MARKET,
                    variety = kite.VARIETY_CO,
                    product=kite.PRODUCT_CO,)
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'stop_trading']= True
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'last_updated_on']= datetime.date.today()
            getquant()
            joblib.dump(qty_df,'./qty_df.p')
            stopbuy = False
        except Exception as e:
            stopsell = True
            print(e)
            getquant()
        placing_order = False
############### CHK QTY #####################################
def ckqnt (tkn,orderquant,trigger_price):
    '''ckqnt (tkn,orderquant,trigger_price)'''
    global placing_order
    myquantity=qty_df.loc[qty_df['instrument_token'] == tkn]['my_qty'].to_numpy()[0]
    quantdiff = orderquant - myquantity
    if(placing_order==False and quantdiff!=0):
        placeneworder(inst_token=tkn,quantdiff=quantdiff,trigger_price=trigger_price)
    return myquantity

In [ ]:
############# Start Trading ###############################
import logging
import threading
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)

counter = 0

def on_ticks(ws, ticks):
    global counter
    counter = counter+1
    print('==========================')
    this_min = datetime.datetime.now().minute
    this_hour = datetime.datetime.now().hour

    for one_stk_data in ticks:
        print('Symbol : ' + str(qty_df.loc[qty_df['instrument_token'] ==
                                           one_stk_data['instrument_token']]
                                ['tradingsymbol'].to_numpy()[0]))
        print('LTP : ' + str(one_stk_data['last_price']))
        
        this_ltp = one_stk_data['last_price']
        
        this_morning_low = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['morning_low'].to_numpy()[0]
        this_morning_high = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['morning_high'].to_numpy()[0]
        this_stk_qty = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['trade_qty'].to_numpy()[0]
        this_stop_trading = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['stop_trading'].to_numpy()[0]
        last_ltp = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['ltp'].to_numpy()[0]
        
        qty_df.at[qty_df.index[qty_df['instrument_token'] ==
                               one_stk_data['instrument_token']].tolist()[0],'ltp'] = this_ltp
        
        if (nifty_direction == 'sell'):
            print('Morning Low : ' + str(this_morning_low))
        else:
            print('Morning High : ' + str(this_morning_high))
        print('*****************************')

        if (this_hour <= 10 and this_min <= 45 and this_stk_qty != 0 and this_stop_trading==False):
            if (nifty_direction == 'sell' and this_ltp < this_morning_low and last_ltp >= this_morning_low):
                ckqnt(one_stk_data['instrument_token'], -this_stk_qty,
                      this_morning_high)
            if (nifty_direction == 'buy' and this_ltp > this_morning_high and last_ltp <= this_morning_high):
                ckqnt(one_stk_data['instrument_token'], this_stk_qty,
                      this_morning_low)
    if(counter%10==0):
        qty_df.to_csv('./live_data.csv')
        joblib.dump(qty_df,'./qty_df.p')
        
    print('==========================')
    print()

def on_connect(ws, response):
    ws.subscribe(qty_df.instrument_token.to_list())
    ws.set_mode(ws.MODE_FULL, qty_df.instrument_token.to_list())

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [ ]:
# kite.place_order(
#     tradingsymbol = 'ONGC',
#     transaction_type =kite.TRANSACTION_TYPE_BUY,
#     trigger_price = 70.00,
#     quantity = 1,
#     order_type=kite.ORDER_TYPE_MARKET,
#     variety=kite.VARIETY_CO,
#     exchange=kite.EXCHANGE_NSE,
#     product=kite.PRODUCT_CO)

In [ ]:
# inst = kite.instruments()
# leninst = len(inst)
# n50list = []
# for i in range(leninst):
#     for j in range(len(nifty50)):
#         if(inst[i]['tradingsymbol']==nifty50[j] and inst[i]['exchange']=='NSE'):
#             n50list.append(inst[i])
# n50list
# mydf = pd.DataFrame(n50list)  
# mydf.to_csv('n50.csv')  